In [27]:
import sys
import os
import torch
import torch.nn as nn
CONTENT_PATH = '/home/yifan/dataset/case_study_mnist_backdoor'
sys.path.append(CONTENT_PATH)
import Model.model as subject_model
model = eval("subject_model.{}()".format('resnet18'))

# 加载预训练模型

model.eval()

train_data = torch.load(os.path.join(CONTENT_PATH, 'Training_data','training_dataset_data.pth'))
train_labels = torch.load(os.path.join(CONTENT_PATH, 'Training_data','training_dataset_label.pth'))
img_tensor = train_data[:1]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
img_tensor = img_tensor.to(device)
model = model.to(device)
print(img_tensor.shape)
# # 设置epsilon
epsilon = 0.01

# # 计算图像对模型的梯度
img_tensor.requires_grad = True
output = model(img_tensor)
label = output.max(1)[1]
loss = nn.CrossEntropyLoss()(output, label)
loss.backward()

# # 应用FGSM算法
img_adv = img_tensor + epsilon * img_tensor.grad.sign()
img_adv = torch.clamp(img_adv, 0, 1)

# # check adv
output_adv = model(img_adv)
predicted_label = output_adv.max(1)[1]
print("Original label:", label.item())
print("Adversarial label:", predicted_label.item())


torch.Size([1, 1, 28, 28])
Original label: 0
Adversarial label: 0
